In [ ]:
# this sample notebook demonstrates the various capabilities 
# of the GRID platform that will allow the user to rapidly solve a 
# simple use case of moving towards a sphere and asking questions
# about it. This could be extrapolated to scenatios like warehouse
# supervision , fault detection in electric lines , etc.

from grid.robot.airgen_drone import AirGenDrone
airgen_drone_0 = AirGenDrone(**{'geo': False})

In [ ]:
from grid.model.perception.vlm.llava import LLaVA
from grid.model.perception.detection.gdino import GroundingDINO
from grid.model.navigation.visualservoing import VisualServoing
detection_model = GroundingDINO()
qa_model = LLaVA()
servoing_model = VisualServoing()

In [ ]:
import airgen
import numpy as np
airgen_drone_0 = AirGenDrone(**{'geo': False})

client = airgen_drone_0.client
client.reset()
client.confirmConnection()
client.enableApiControl(True)
client.armDisarm(True)

# Take off
client.takeoffAsync().join()

client.moveByVelocityAsync(5, 0, 0, 5).join()
client.moveByVelocityAsync(0, 2, 0, 5).join()


# Capture image from AirGen
def capture_images(client):
    responses = client.simGetImages([
        airgen.ImageRequest("0", airgen.ImageType.Scene, False, False),
        airgen.ImageRequest("0", airgen.ImageType.DepthPerspective, True, False)
    ])
    rgb_image = np.frombuffer(responses[0].image_data_uint8, dtype=np.uint8).reshape(responses[0].height, responses[0].width, 3)
    depth_image = np.array(responses[1].image_data_float, dtype=np.float32).reshape(responses[1].height, responses[1].width)
    return rgb_image, depth_image

# Calculate distance to the object
def calculate_distance(depth_image, bounding_box):
    x_min, y_min, x_max, y_max = bounding_box
    depth_region = depth_image[int(y_min):int(y_max), int(x_min):int(x_max)]
    distance = np.min(depth_region)  # minimum distance within the bounding box
    return distance

# Sample use case
def move_towards_sphere_and_ask_color(client, detection_model, servoing_model, qa_model, target_object, question, stop_distance=10.0, iterations=30):
    for i in range(iterations):
        # Step 1: Capture images
        rgb_image, depth_image = capture_images(client)
        
        # Step 2: Detect objects
        bounding_boxes, phrases = detection_model.detect_object(rgb_image, target_object)
        
        if len(bounding_boxes) == 0:
            print(f"No objects detected for prompt: {target_object}")
            continue
        
        # Step 3: Select the first detected spherical object
        target_bbox = bounding_boxes[0]
        target_phrase = phrases[0]
        
        # Calculate the center of the bounding box
        x_center = (target_bbox[0] + target_bbox[2]) / 2
        y_center = (target_bbox[1] + target_bbox[3]) / 2
        target_image_coord = (x_center, y_center)
        
        # Calculate the distance to the target object
        distance = calculate_distance(depth_image, target_bbox)
        print(f"Iteration {i+1}: Distance to target: {distance:.2f} meters")
        
        # Step 4: Check if the drone is within the stop distance
        if distance <= stop_distance:
            print(f"Stopping: Within {stop_distance} meters of the target object")
            break
        
        # Step 5: Move drone to the target
        camera_param = {
            "width": rgb_image.shape[1],
            "height": rgb_image.shape[0],
            "fov": 90.0,
            "camera_orientation_euler_pry": (0.0, 0.0, 0.0)
        }
        delta_yaw, velocity_vector = servoing_model.moveDrone2Target(target_image_coord, camera_param)
        client.moveByVelocityAsync(velocity_vector[0], velocity_vector[1], velocity_vector[2], 1).join()
        
        print(f"Iteration {i+1}: Moving towards {target_phrase}")
        
    # Step 6: Capture image again to ask the question about the sphere's color
    rgb_image, _ = capture_images(client)
    
    # Step 7: Answer question about the color of the sphere
    answer = qa_model.run(rgb_image, question)
    print(f"Answer to '{question}': {answer}")

# Use case parameters
target_object = "sphere"
question = "What is the color of the sphere?"

# Run the workflow to move towards the sphere and ask about its color
move_towards_sphere_and_ask_color(client, detection_model, servoing_model, qa_model, target_object, question, stop_distance=5.0, iterations=30)

# Land the drone
client.landAsync().join()
client.armDisarm(False)
client.enableApiControl(False)
